Shows how to generate and visualize emissions data for the GRAL model.

In [1]:
# Small test inventory for saving to gral format
import logging
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, MultiLineString
from emiproc.inventories import Inventory, EmissionInfo
from emiproc.tests_utils.test_inventories import inv_with_pnt_sources
from emiproc.tests_utils import TEST_OUTPUTS_DIR
from emiproc.exports.gral import export_to_gral

gral_test_inv = inv_with_pnt_sources.copy()
# Addintg some shapes that should be supported
gral_test_inv.gdfs["adf"] = gpd.GeoDataFrame(
    {
        "CO2": [1, 2, 1, 3, 1],
        "CH4": [3, 1, 1, 1, 1],
    },
    geometry=[
        LineString([(0.75, 0.75), (1.25, 1.25), (2.25, 1.25)]),
        Polygon([(0.25, 0.25), (0.75, 0.75), (0.25, 0.75)]),
        MultiPolygon(
            [
                # Square from 2 to 4 
                Polygon([(2, 2), (4, 2), (4, 4), (2, 4)]),
                # Sqare from 5 to 6
                Polygon([(5, 5), (6, 5), (6, 6), (5, 6)]),
                # A triangle from 8 to 12 
                Polygon([(8, 8), (12, 8), (12, 12)]),
            ]
        ),
        MultiLineString(
            [
                LineString([(0.25, 0.25), (0.75, 0.75), (3.25, 1.25)]),
                LineString([(0.75, 0.75), (0.5, 0.25)]),
            ]
        ),
        Point(0.5, 0.5),
    ],
)

from pygg.grids import GralGrid

inv = gral_test_inv

grid = GralGrid(
    dz=1.5,
    stretch=0.5,
    nx=20,
    ny=20,
    nslice=20,
    sourcegroups="",
    xmin=0,
    xmax=10,
    ymin=0,
    ymax=4,
    latitude=0,
    crs=inv.crs,
)
grid.building_heights = np.zeros((grid.ny, grid.nx))

inv.emission_infos = {
    "adf": EmissionInfo(),
    "blek": EmissionInfo(),
    "liku": EmissionInfo(),
    "other": EmissionInfo(),
    "test": EmissionInfo(),
}

export_to_gral(inv, grid, TEST_OUTPUTS_DIR, 1)


/users/lconstan/.local/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


Visualize the generated outputs

In [2]:
from emiproc.tests_utils import TEST_OUTPUTS_DIR
from emiproc.inventories.gral import GralInventory
from emiproc.grids import LV95

emissions_dir = TEST_OUTPUTS_DIR
inv = GralInventory(emissions_dir)

/users/lconstan/emiproc/emiproc/inventories/gral.py:224: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  line_lenghts = gs_lines.length * 1e-3
/users/lconstan/emiproc/emiproc/inventories/gral.py:224: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  line_lenghts = gs_lines.length * 1e-3


In [3]:
from emiproc.plots import explore_inventory

In [4]:
explore_inventory(inv, inv.categories[0], inv.substances[0])